In [ ]:

!git clone https://github.com/MaastrichtU-IDS/LMKBC-2023.git

In [ ]:
!git reset --hard
!git pull 

In [ ]:
!git pull

In [ ]:
import os
os.chdir('LMKBC-2023/')

In [3]:
!pip install accelerate -U
!pip install transformers
!pip3 install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: cudf 23.2.0 has a non-standard dependency specifier cuda-python>=12.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of cudf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pylibraft 23.2.0 has a non-standard dependency specifier cuda-python>=12.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pylibraft or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: rmm 23.2.0 has a non-standard dependency specifier cuda-python>=12.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of rmm 

In [ ]:
!python src/run.py --mode "train_fm"



In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
!git add . 
!git commit -m 'from server' 
!git push


In [ ]:
#!model_load_dir = '/workspace/persistent/LMKBC-2023/bin/pretrain_fill_mask/bert-large-cased/best_ckpt'
!model_load_dir ='/workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased/best_ckpt'

!python  /workspace/persistent/LMKBC-2023/src/fm_model.py  --test_fn /workspace/persistent/LMKBC-2023/data/val.jsonl --template_fn res/prompts0.csv   --output_fn /workspace/persistent/LMKBC-2023/output/filled-mask-valid.jsonl --train_fn /workspace/persistent/LMKBC-2023/data/train.jsonl          --train_batch_size 256 --gpu 0  --top_k 30 --threshold 0.1  --dev_fn  /workspace/persistent/LMKBC-2023/data/train_tiny.jsonl         --mode "tr ain test"  --train_epoch 50 --learning_rate 5e-5         --model_load_dir   $model_load_dir         --model_save_dir  /workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased        --model_best_dir  /workspace/persistent/LMKBC-2023/bin/train_full/fill_mask/bert-large-cased/best_ckpt 

In [ ]:
   python {config.SRC_DIR}/fm_model.py  --test_fn {test_fn} --template_fn res/prompts0.csv  --output_fn {OUTPUT_FILE} --train_fn {config.DATA_DIR}/train.jsonl --train_batch_size 64 --gpu {args.gpu}  --top_k 30 --threshold 0.1  --dev_fn  {config.DATA_DIR}/train_tiny.jsonl --mode "train test" --train_epoch 50 --learning_rate 5e-5 --model_load_dir {last_best_dir} --model_save_dir {fm_model_save_dir} --model_best_dir  {fm_model_best_dir}

In [ ]:
!nvidia-smi

In [ ]:
!cd /workspace/persistent/LMKBC-2023/
import os

command = '''
nohup python src/run.py --label 'mask-random' --task pfm,fm --load_dir bert-base-cased --epoch 20,50 --mode 'random,' &
'''
os.system(command)

command = '''
nohup python src/run.py --label 'mask-single' --task pfm,fm --load_dir bert-base-cased --epoch 20,50 --mode 'single,' &
'''
os.system(command)

command = '''
nohup python src/run.py --label 'mask-fold' --task pfm,fm --load_dir bert-base-cased --epoch 20,50 --mode 'fold,' &
'''
os.system(command)

In [ ]:
import src.config 

label = 'mask-random'
pfm_model_save_dir =f"bin/{label}/pfm"
pfm_model_best_dir = pfm_model_save_dir + "/best_ckpt"
input_fp = f"{config.RES_DIR}/wikidata/Country-Language-State/filter.json"
model_load_dir = config.bert_base_cased
p

cmd_pretrain_filled_mask = f"""

python src/pre_model.py   --train_fn {input_fp}  --train_batch_size 32 --gpu 0  --train_epoch 32 --learning_rate 5e-5  --mask_strategy random 
 --model_load_dir {model_load_dir} --model_save_dir {pfm_model_save_dir} --model_best_dir  {pfm_model_best_dir}

"""

os.system(cmd_pretrain_filled_mask)


OUTPUT_FILE = f'output/filled-mask-valid.jsonl'

fm_model_save_dir =f"bin/label/fm"
fm_model_best_dir = fm_model_save_dir + "/best_ckpt"
test_fn = config.VAL_FN

cmd_run_fillmask = f"""

python src/fm_model.py  --test_fn data/val.jsonl --template_fn res/prompts0.csv  --output_fn {OUTPUT_FILE} --train_fn data/train.jsonl 
  --train_batch_size 64 --gpu 0  --top_k 30 --threshold 0.1  --dev_fn  data/train_tiny.jsonl --mode "train test"   
  --train_epoch 50 --learning_rate 5e-5 --model_load_dir {pfm_model_best_dir} --model_save_dir {fm_model_save_dir} 
  --model_best_dir  {fm_model_best_dir}

"""

os.system(cmd_run_fillmask)

    

In [ ]:
!cat nohup.out 

In [ ]:
  !python /workspace/persistent/LMKBC-2023/src/fm_model.py  --test_fn /workspace/persistent/LMKBC-2023/data/val.jsonl --template_fn res/prompts0.csv  --output_fn /workspace/persistent/LMKBC-2023/output/filled-mask-valid.jsonl --train_fn /workspace/persistent/LMKBC-2023/data/train.jsonl --train_batch_size 64 --gpu 0  --top_k 30 --threshold 0.1  --dev_fn  /workspace/persistent/LMKBC-2023/data/train_tiny.jsonl --mode "train test" --train_epoch 50 --learning_rate 5e-5 --model_load_dir /workspace/persistent/LMKBC-2023/bin/default/pretrain_fill_mask/best_ckpt --model_save_dir /workspace/persistent/LMKBC-2023/bin/default/fill_mask --model_best_dir  /workspace/persistent/LMKBC-2023/bin/default/fill_mask/best_ckpt

In [ ]:
!tail -n 1000  nohup.out| grep loss

In [7]:
import os
import json 
print('cwd', os.getcwd())

import src.util


result_fp = 'res/results.jsonl'
sentence_list =  open(result_fp).readlines()
json_list = [json.loads(s) for s in sentence_list] 

result_json = json.loads(open(result_fp).readlines()[-10])
label = result_json['args']['model_save_dir']
print(label)
need_keys = ['CountryBordersCountry','CountryHasOfficialLanguage',
             'CountryHasStates',"StateBordersState",'Average']
result = result_json['metric']
#print(result.keys())
for k in need_keys:
    print(k, json.dumps(result[k],indent = 2))


cwd /workspace/project/LMKBC-2023
getcwd /workspace/project/LMKBC-2023


JSONDecodeError: Extra data: line 1 column 2992 (char 2991)